In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import set_visible_GPUs, reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


import tf_lab.autopredictors.scripts.virt_scan_data as vscan

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch
        
from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
complete_n_samples = 4096
incomplete_n_samples = 2048

# experiment_name = '10_vscan_emd_' + str(incomplete_n_samples) + '_' + str(complete_n_samples) + 'pts'
experiment_name = 'scratch'

do_training = True
do_plotting = False
first_time_running = True
load_model = False
load_test_data = True

tr_val_te = [0.80, 0.10, 0.10] # Percentages of train-test-validation tests

class_type = raw_input('Give me the class type.\n').lower()
class_syn_id = shape_net_category_to_synth_id()[class_type]


top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'

v_scan_top_dir = osp.join(top_data_dir, 'From_Matthias/shapenet_dim32_sdf_pc', class_syn_id)

complete_pclouds_path = osp.join(top_data_dir, 'ShapeNetPointClouds/from_manifold_meshes/', \
                             str(complete_n_samples), class_syn_id)

Give me the class type.
chair


In [4]:
seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

n_input = [incomplete_n_samples, 3]
n_output = [complete_n_samples, 3]

train_dir = osp.join(top_data_dir, 'OUT/models/incomplete_pclouds/')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/scratch'

In [5]:
cmpl_file_names = pio.load_filenames_of_input_data(complete_pclouds_path, points_extension)
cmpl_pclouds, cmpl_model_names, syn_ids = pio.load_crude_point_clouds(file_names=cmpl_file_names, n_threads=25)
assert(len(np.unique(class_ids)) == 1)
print '%d files containing complete point clouds with normals were found.' % (len(cmpl_pclouds), )

6778 files containing complete point clouds with normals were found.


In [10]:
train_data_, val_data_, test_data_, r_indices = pio.train_validate_test_split(
                                                            [cmpl_pclouds, cmpl_model_names],
                                                            train_perc=tr_val_te[0],
                                                            validate_perc=tr_val_te[1],
                                                            test_perc=tr_val_te[2],
                                                            seed=seed)
if first_time_running:
    model_ids = np.vstack((cmpl_model_names, syn_ids)).T
    write_model_ids_of_datasets(train_dir, model_ids, r_indices)

In [18]:
search_pattern = '.ply'
incomplete_pcloud_files = pio.load_filenames_of_input_data(v_scan_top_dir, '.ply')


    


# train_data = PointCloudDataSet(train_data_[0], labels=train_data_[1])
# val_data = PointCloudDataSet(val_data_[0], labels=val_data_[1])
# test_data = PointCloudDataSet(test_data_[0], labels=test_data_[1])


# pio.train_validate_test_split(cmpl_model_names )

48000

In [ ]:
search_pattern = '.ply'
   
if do_training:
    train_id_file = osp.join(top_data_dir, 'From_Matthias/train_test_models/train_models_with_scan_id.txt')
    
    train_data = vscan.load_single_class_incomplete_dataset(v_scan_top_dir, train_id_file, class_syn_id,
                                                      full_pclouds, full_model_names, n_threads=25,
                                                      n_samples=incomplete_n_samples, search_pattern=search_pattern)

if load_test_data:
    test_id_file = osp.join(top_data_dir, 'From_Matthias/train_test_models/test_models_with_scan_id.txt')
    test_data = vscan.load_single_class_incomplete_dataset(v_scan_top_dir, test_id_file, class_syn_id, 
                                                      full_pclouds, full_model_names, n_threads=25,
                                                      n_samples=incomplete_n_samples, search_pattern=search_pattern)